In [1]:
import os
import re

# Modifications

### Load data:

In [2]:
# !wget https://cv.snu.ac.kr/research/EDSR/benchmark.tar
# !wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
# !wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip

### Unzip downloads:

In [3]:
# %mkdir div2k
# !unzip -q DIV2K_train_HR.zip -d div2k
# !unzip -q DIV2K_train_LR_bicubic_X2.zip -d div2k
# !tar -xf benchmark.tar

### Make symlinks (necessary for BasicSR):

In [4]:
# root = !pwd
# root = root[0]

# %mkdir basicsr/datasets/DIV2K
# !ln -s $root/div2k/*  $root/basicsr/datasets/DIV2K
# !ln -s  $root/benchmark  $root/basicsr/datasets

### Make patches:

In [5]:
# %cd basicsr
# !python3 ../scripts/data_preparation/extract_subimages.py
# %cd ..

### Change parameters:

In [6]:
train_params = {
    'num_gpu': 0,
    'num_worker_pre_gpu': 4,
    'batch_size_per_gpu': 16,
    'dataset_enlarge_ratio': 1,
    'total_iter': 10,
    'val_freq': '!!float 1e1',
    'save_checkpoint_freq': '!!float 1e1',
    'print_freq': 10,
    'use_tb_logger': 'false'
}

In [7]:
test_params = {
    'num_gpu': 0
}

In [8]:
def get_model_name(filename):
    parts = filename.split('.')[0].split('_')[1:]
    new_parts = [parts[0]]
    for part in parts[1:]:
        if part.lower() == 'se':
            new_parts.append('SE') 
            continue
        new_parts.append(part.capitalize()) 
    return "".join(new_parts)

In [9]:
root = os.path.abspath('.')
train_path = 'options/train/'

for file in os.listdir(train_path):
    full_path = root + '/' + train_path + file
    text = None
    with open(full_path, 'r') as f:
        text = f.read()
        text = re.sub('dataroot_gt: [a-zA-Z0-9-/]+/datasets/', 'dataroot_gt: ' + root + '/basicsr/datasets/', text)
        text = re.sub('dataroot_lq: [a-zA-Z0-9-/]+/datasets/', 'dataroot_lq: ' + root + '/basicsr/datasets/', text)
        for param, value in train_params.items():
            text = re.sub(param + ':' + '.*\n', param + ': ' + str(value) + '\n', text)
    text = re.sub('name:' + '.*\n', 'name: ' + file.split('.')[0] + '\n', text, count=1)
    text = re.sub('network_g:\n  type: ' + '.*\n', 'network_g:\n  type: ' + get_model_name(file) + '\n', text, count=1)
    with open(full_path, 'w') as f:
        f.write(text)

In [10]:
root = os.path.abspath('.')
test_path = 'options/test/'

for file in os.listdir(test_path):
    full_path = root + '/' + test_path + file
    text = None
    with open(full_path, 'r') as f:
        text = f.read()
        text = re.sub('dataroot_gt: [a-zA-Z0-9-/]+/datasets/', 'dataroot_gt: ' + root + '/basicsr/datasets/', text)
        text = re.sub('dataroot_lq: [a-zA-Z0-9-/]+/datasets/', 'dataroot_lq: ' + root + '/basicsr/datasets/', text)
        for param, value in test_params.items():
            text = re.sub(param + ':' + '.*\n', param + ': ' + str(value) + '\n', text)
    text = re.sub('name:' + '.*\n', 'name: ' + file.split('.')[0] + '\n', text, count=1)
    text = re.sub('network_g:\n  type: ' + '.*\n', 'network_g:\n  type: ' + get_model_name(file) + '\n', text, count=1)
    text = re.sub(
        'pretrain_network_g: .*\n', 
        'pretrain_network_g: ' + root + '/experiments/' + \
            file.replace('test', 'train').split('.')[0] + '/models/net_g_latest.pth' + '\n', 
        text
    )
    with open(full_path, 'w') as f:
        f.write(text)

## Experiments:

In [12]:
%cd basicsr/

/home/kovcharenko51/2023-Project-137/code/basicsr


### EDSR

In [ ]:
!python3 train.py -opt ../options/train/train_EDSR.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_EDSR.yaml

### Binary EDSR

In [13]:
!python3 train.py -opt ../options/train/train_BinEDSR.yaml

Disable distributed.
2023-03-31 15:06:39,810 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.4.2
	PyTorch: 1.12.1+cu116
	TorchVision: 0.13.1+cu116
2023-03-31 15:06:39,810 INFO: 
  name: train_BinEDSR
  model_type: SRModel
  scale: 2
  num_gpu: 0
  manual_seed: 0
  datasets:[
    train:[
      name: DIV2K
      type: PairedImageDataset
      dataroot_gt: /home/kovcharenko51/2023-Project-137/code/basi

2023-03-31 15:06:39,939 INFO: Start training from epoch: 0, iter: 0


In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR.yaml

### SE-attention on binary branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_none_se.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_none_se.yaml

### Spatial attention on binary branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_none_spat.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_none_spat.yaml

### Pixel attention on binary branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_none_pixel.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_none_pixel.yaml

### SE-attention on residual branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_se_none.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_se_none.yaml

### Spatial attention on residual branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_spat_none.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_spat_none.yaml

### Pixel attention on residual branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_pixel_none.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_pixel_none.yaml

### SE-attention on both branches

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_se_se.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_se_se.yaml

### SE-attention on residual branch, spatial attention on binary branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_se_spat.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_se_spat.yaml

### SE-attention on residual branch, pixel attention on binary branch

In [ ]:
!python3 train.py -opt ../options/train/train_BinEDSR_se_pixel.yaml

In [ ]:
!python3 test.py -opt ../options/test/test_BinEDSR_se_pixel.yaml